In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import textwrap

# Load the uploaded Excel file to inspect its structure
file_path = 'ating-out menu.xlsx'
excel_data = pd.ExcelFile(file_path)

# Define Monte Carlo simulation function
def monte_carlo_simulation(data, sample_percentage=0.2, n_simulations=1000):
    results = []
    num_to_sample = int(len(data) * sample_percentage)
    for _ in range(n_simulations):
        sampled_data = data.sample(n=num_to_sample, replace=False)
        noise = np.random.uniform(0.9, 1.1, size=sampled_data.shape)
        noisy_data = sampled_data * noise
        mean_values = noisy_data.mean()
        results.append(mean_values)
    return pd.DataFrame(results)

# Define Japanese to English sheet name mapping
sheet_name_mapping = {
    '1. 日本そば': 'Japanese Noodles',
    '2. 中華そば': 'Chinese Noodles',
    '3. 他の麺類外食': 'Other Noodles',
    '4. すし(外食)': 'Sushi',
    '5. 和食': 'Japanese Restaurant',
    '6. 中華食': 'Chinese Restaurant',
    '7. 洋食': 'Western Restaurant',
    '8. 焼肉': 'BBQ',
    '9. ハンバーグ': 'Burgers',
    '10. 他の主食的外食': 'Others',
    '11. 喫茶代': 'Cafe',
    '12. 飲酒代': 'Bar'
}

# Perform Monte Carlo simulation
simulation_results = {}
for sheet_name in excel_data.sheet_names:
    data = pd.read_excel(file_path, sheet_name=sheet_name)
    numeric_data = data.select_dtypes(include=[np.number])
    simulated_means = monte_carlo_simulation(numeric_data, sample_percentage=0.20, n_simulations=1000)
    english_sheet_name = sheet_name_mapping.get(sheet_name, sheet_name)
    simulation_results[english_sheet_name] = simulated_means

# Combine data for visualization
combined_data = pd.DataFrame()
for sheet_name, df in simulation_results.items():
    df['Category'] = sheet_name  # Add English category name
    combined_data = pd.concat([combined_data, df], ignore_index=True)

# Set the font style to Arial and color to black
plt.rcParams["font.family"] = "Arial"
plt.rcParams["text.color"] = "black"

# Define subplots layout (2 columns, adjust to fit A4 size)
fig, axes = plt.subplots(3, 2, figsize=(8.27, 10))  # A4 size in inches

# Flatten axes for easy iteration
axes = axes.flatten()

# Nutritional factors to plot
nutritional_columns = ['Price (JPY)', 'Energy (kcal)', 'Protein (g)', 'Fat (g)', 'Carbohydrates (g)', 'Salt equivalent (g)']
for i, nutrient in enumerate(nutritional_columns):
    ax = axes[i]
    sns.boxplot(
        data=combined_data, 
        x='Category', 
        y=nutrient, 
        ax=ax, 
        linewidth=0.8,  # Set thinner boxplot line
        showfliers=False  # Hide outliers
    )
    ax.set_title(f"{nutrient}")
    ax.set_ylabel(nutrient)
    ax.set_xticks([])  # Remove x-axis labels for all but the last row
    ax.set_xlabel("")

# Add x-axis ticks and labels only to the last row
categories = combined_data['Category'].unique()

# Wrap long labels
wrapped_categories = [textwrap.fill(cat, width=10) for cat in categories]  # Adjust `width` to control wrap length

for ax in axes[-2:]:  # Adjust for the last two plots
    ax.set_xticks(range(len(categories)))  # Set fixed positions for ticks
    ax.set_xticklabels(wrapped_categories, rotation=90, fontsize=9)  # Horizontal labels, smaller font

# Remove extra subplot if less than 6 plots
for j in range(len(nutritional_columns), len(axes)):
    fig.delaxes(axes[j])

# Adjust layout to fit A4
fig.tight_layout()

# Save the figure as high-quality SVG
plt.savefig("Nutritional_Distribution.svg", format="svg", dpi=300)

# Show the result
plt.show()

In [ ]:
# Load the additional data
age_income_file = 'FIES_income_age.xlsx'

# Read data from 'age' and 'income' sheets
age_data = pd.read_excel(age_income_file, sheet_name='age')
income_data = pd.read_excel(age_income_file, sheet_name='income')

age_income_combined = combined_data.merge(age_data, on='Category', how='left')
age_income_combined = age_income_combined.merge(income_data, on='Category', how='left')

# Calculate nutrition per spending
nutritional_columns = ['Energy (kcal)', 'Protein (g)', 'Fat (g)', 'Carbohydrates (g)', 'Salt equivalent (g)']
for nutrient in nutritional_columns:
    age_income_combined[f"{nutrient} per JPY"] = age_income_combined[nutrient] / age_income_combined['Price (JPY)']

In [ ]:
age_columns = [col for col in age_income_combined.columns if '歳' in col and '平均価格' in col]
income_columns = [col for col in age_income_combined.columns if '万円' in col and '平均価格' in col]

age_income_combined['Price (100 JPY)'] = age_income_combined['Price (JPY)'] / 100
for nutrient in nutritional_columns:
    age_income_combined[f"{nutrient} per 100 JPY"] = age_income_combined[f"{nutrient} per JPY"] * 100

results_by_age_category = {}
results_by_income_category = {}

for age_col in age_columns:
    age_group = age_col.split('・')[0]
    avg_price_by_age = age_income_combined[age_col]
    nutrition_by_age_category = {}
    
    for category in age_income_combined['Category'].unique():
        category_data = age_income_combined[age_income_combined['Category'] == category]
        nutrition_per_meal = {}
        
        for nutrient in nutritional_columns:
            nutrition_per_meal[nutrient] = (avg_price_by_age / 100 * category_data[f"{nutrient} per 100 JPY"]).mean()
        
        nutrition_by_age_category[category] = nutrition_per_meal
    
    results_by_age_category[age_group] = nutrition_by_age_category

for income_col in income_columns:
    income_group = income_col.split('・')[0]  
    avg_price_by_income = age_income_combined[income_col]
    nutrition_by_income_category = {}
    
    for category in age_income_combined['Category'].unique():
        category_data = age_income_combined[age_income_combined['Category'] == category]
        nutrition_per_meal = {}
        
        for nutrient in nutritional_columns:
            nutrition_per_meal[nutrient] = (avg_price_by_income / 100 * category_data[f"{nutrient} per 100 JPY"]).mean()
        
        nutrition_by_income_category[category] = nutrition_per_meal
    
    results_by_income_category[income_group] = nutrition_by_income_category

age_results = []
for age_group, category_data in results_by_age_category.items():
    for category, nutrients in category_data.items():
        row = {'Age Group': age_group, 'Category': category}
        row.update(nutrients)
        age_results.append(row)

income_results = []
for income_group, category_data in results_by_income_category.items():
    for category, nutrients in category_data.items():
        row = {'Income Group': income_group, 'Category': category}
        row.update(nutrients)
        income_results.append(row)

results_age_category_df = pd.DataFrame(age_results)
results_income_category_df = pd.DataFrame(income_results)

In [ ]:
category_stats = combined_data.groupby('Category').agg(
    avg_price=('Price (JPY)', 'mean'),
    energy_per_100_jpy=('Energy (kcal)', lambda x: (x / combined_data['Price (JPY)']).mean() * 100),
    protein_per_100_jpy=('Protein (g)', lambda x: (x / combined_data['Price (JPY)']).mean() * 100),
    fat_per_100_jpy=('Fat (g)', lambda x: (x / combined_data['Price (JPY)']).mean() * 100),
    carbs_per_100_jpy=('Carbohydrates (g)', lambda x: (x / combined_data['Price (JPY)']).mean() * 100),
    salt_per_100_jpy=('Salt equivalent (g)', lambda x: (x / combined_data['Price (JPY)']).mean() * 100)
).reset_index()

category_stats